# This notebook demos a diffusion stencil

Here follows the diffusion stencil

In [1]:
from dusk.script import *


@stencil
def diffusion(
    TV: Field[Vertex],
    TE: Field[Edge], TEinit: Field[Edge], TE_t: Field[Edge], TEnabla2: Field[Edge],
    inv_primal_edge_length: Field[Edge], inv_vert_vert_length: Field[Edge], nnbhV: Field[Vertex],
    boundary_edge: Field[Edge],
    kappa: Field[Edge], dt: Field[Edge]
) -> None:

    with levels_upward:
        # initialize
        TEinit = TE
    
        # predict
        TE = TEinit + 0.5*dt*TE_t

        # interpolate temperature from edges to vertices
        TV = sum_over(Vertex > Edge, TE) / nnbhV

        # compute nabla2 using the finite differences
        TEnabla2 = sum_over(
            Edge > Cell > Vertex,
            4.0 * TV,
            weights=[
                inv_primal_edge_length ** 2.,
                inv_primal_edge_length ** 2.,
                inv_vert_vert_length ** 2.,
                inv_vert_vert_length ** 2.,
            ],
        )
        TEnabla2 = TEnabla2 - (
            (8.0 * TE * inv_primal_edge_length ** 2.)
            + (8.0 * TE * inv_vert_vert_length ** 2.)
        )
    
        # build ODEs
        if (boundary_edge):
            TE_t = 0.
        else:
            TE_t = kappa*TEnabla2
    
        # correct
        TE = TEinit + dt*TE_t


Then we can use dusk's Python API to convert the stencils to SIR. This API can also invoke dawn to compile SIR to C++ which we will write to `diffusion_cxx-naive.cpp`:

In [2]:
from dusk.transpile import callable_to_pyast, pyast_to_sir, sir_to_cpp

with open("diffusion_cxx-naive.cpp", "w+") as f:
    f.write(sir_to_cpp(pyast_to_sir(callable_to_pyast(diffusion))))

The generated C++ code also requires a driver which is already setup for this demo. With the driver code we can generate an executable `runner`:

In [3]:
!make

First, we put the runner into test mode to ensure that the computed kernel is correct:

In [4]:
!./runner test

mesh stats: #cells 680 #nodes 378 #edges 1057
looks like something is off... please recheck your stencil

If the tester reported that your dusk stencil works correctly, you can now run the complete stencil and visualize

In [5]:
!./runner run

mesh stats: #cells 680 #nodes 378 #edges 1057
time 0.105 timestep 1 dt 0.005
time 0.11 timestep 2 dt 0.005
time 0.115 timestep 3 dt 0.005
time 0.12 timestep 4 dt 0.005
time 0.125 timestep 5 dt 0.005
time 0.13 timestep 6 dt 0.005
time 0.135 timestep 7 dt 0.005
time 0.14 timestep 8 dt 0.005
time 0.145 timestep 9 dt 0.005
time 0.15 timestep 10 dt 0.005
time 0.155 timestep 11 dt 0.005
time 0.16 timestep 12 dt 0.005
time 0.165 timestep 13 dt 0.005
time 0.17 timestep 14 dt 0.005
time 0.175 timestep 15 dt 0.005
time 0.18 timestep 16 dt 0.005
time 0.185 timestep 17 dt 0.005
time 0.19 timestep 18 dt 0.005
time 0.195 timestep 19 dt 0.005
time 0.2 timestep 20 dt 0.005
time 0.205 timestep 21 dt 0.005
time 0.21 timestep 22 dt 0.005
time 0.215 timestep 23 dt 0.005
time 0.22 timestep 24 dt 0.005
time 0.225 timestep 25 dt 0.005
time 0.23 timestep 26 dt 0.005
time 0.235 timestep 27 dt 0.005
time 0.24 timestep 28 dt 0.005
time 0.245 timestep 29 dt 0.005
time 0.25 timestep 30 dt 0.005
time 0.255 timestep

In [6]:
%%capture
import prep_animation as pa
import matplotlib.animation as animation
from IPython.display import HTML
animate = animation.FuncAnimation(pa.fig, pa.update_plot, len(pa.out_files), fargs=(pa.zarray, pa.plot), interval = 50)

In [7]:
HTML(animate.to_html5_video())